In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Hugging Face

[Hugging Face](huggingface.co) is a Data Science platform that (among many other functions) hosts
- Pre-trained models
- Datasets

They are particularly (but not exclusively) known from Transformers and NLP tasks.

We will use them as our paradigm for Transfer Learning.



Our overview will necessarily be brief.

We recommend the [free, on-line course](https://huggingface.co/course)
- We will not explain the datasets API
    - similar to Tensorflow Dataset API

# NLP: more than just a model

You will find many models for NLP on HuggingFace.

One of the most important aspects of a Model Hub for NLP is
- providing a consistent interface to multiple models
- with different authors

## Pre and post processing

To ensure universality of models obtained from many sources
- all NLP models take sequences of *tokens* as inputs
- **not** sequences of words (the raw input format)

Thus, there needs to be some *pre-processing* performed before using a pre-trained model

Similarly, there may need to be some *post-processing* performed on the model output.

For example: creating a task-specific "head"
-  e.g., classifier over *our task's* classes

In addition to models, HuggingFace
- supplies common pre-processors and post-processors
- wraps them all up into a simple to use, powerful organization

## One model, many flavors

As we shall see, HuggingFace models come in many variations
- `Keras` or `Pytorch`
- with a "head" specific to a common task

This reduces the need for user customization.

HuggingFace also makes it easy to match the correct pre-processor (Tokenizer) to a model
- a model is identified by a "check-point" referring to its training dataset

        checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
- one instantiates a model from a checkpoint

         model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

    - this model has a "head" specialized for the task of Sequence Classification (e.g., Sentiment Analysis)
        - the `ForSequenceClassification` suffix
    - this model is implemented in TensorFlow
        - the `TF` prefix
- it's easy to choose the "correct" Tokenizer by using the same one as was used in training the model
      
        tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## Preparing input examples

A unique aspect to sequence inputs is that the sequence length varies across examples.

Hence, "padding" is usually necessary to make example lengths uniform.

It is tempting equalize the length of each example to the length of the longest example in the *training dataset*.

This is not the best strategy
- the model works in much smaller *mini-batches*
- only necessary to equalize the length of each example *within* a batch
    - not *across* all batches


Equalizing sequence length of examples within a batch is just one example of
preparing examples in a batch.

Another case: masking words (usually randomly) in a example for a Masked Language Modeling task.

HuggingFace has an abstract class [`DataCollator`](https://huggingface.co/docs/transformers/main_classes/data_collator#data-collator) to prepare examples within a batch
- specializations for common tasks

## Putting it all together: running an NLP task

Easy !  So is actually running all the steps needed to generate output

    sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
    
    tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")
    
    output = model(**tokens)

And one "not so easy" part
- Like any classifier: the output is a *vector* of probabilities over the universe of tokens
    - Technically: they are "logits" rather than probabilities
- So the *user* needs to sample from this vector in order to choose a single output token
    - e.g., the one with highest probability
- This maximizes the flexibility for the user
    - to sample rather than choose the token with highest probability
    - to add further logic
        - sample "top 5" tokens
        

**Remember**

Hugging Face standard is to output
- logits
- rather than probabilities

# A model has many "flavors"

In our discussion of Modern NLP, we highlighted the "Text to Text Universal API"
- Can translate many NLP tasks into variations of translating Text to Text
- A post-processing step can map back from the output text to the actual Task outputs

Hugging Face
- implements a "base model" (universal API)
- and the derives specializations with task-specific "heads"

Moreover, Hugging Face models usually have implementations in both TensorFlow and PyTorch.

The name of a Hugging Face model is a compound string indicating
- the base
- the specialized head
- the implementation language

Illustrating this with the DistilBERT model implemented in TensorFlow ([github](https://github.com/huggingface/transformers/blob/main/src/transformers/models/distilbert/modeling_tf_distilbert.py#L24)), here is the class hierarchy:
- `class TFDistilBertPreTrainedModel(TFPreTrainedModel):`
- `class TFDistilBertModel(TFDistilBertPreTrainedModel)`
- `class TFDistilBertForMaskedLM(TFDistilBertPreTrainedModel, TFMaskedLanguageModelingLoss):`
- `class TFDistilBertForSequenceClassification(TFDistilBertPreTrainedModel, TFSequenceClassificationLoss):`
- `class TFDistilBertForTokenClassification(TFDistilBertPreTrainedModel, TFTokenClassificationLoss):`
- $\vdots$

The `TF` prefix indicates that the implementation language is TensorFlow.

- There is an abstract class for Pretrained TensorFlow models: `TFPreTrainedModel)`
- `TFDistilBertPreTrainedModel` is a pretrained model
- There are specialization of `TFDistilBertPreTrainedModel` for each NLP task
    - `TFDistilBertForMaskedLM` for the Masked Language Modelin task
    - `TFDistilBertForSequenceClassification` for the Sequence Classification task
    - `TFDistilBertForTokenClassification` for the Token Classification task

You get to choose !
- If you choose a model instance with the `*For*` pattern in the name
    - you get a pre-trained model with a Classification head (post-processing) for a specific task
- If you choose a model instances *without* the `*For*` pattern
    - you get a model without a head (or post-processing)
    - you can add your own
    
You can also instantiate a model *without* pre-trained weights
- just the architecture

# Example: `TFDistilBertForSequenceClassification`

Let's take a high-level look at one specific model: `TFDistilBertForSequenceClassification`
- this is a TensorFlow implementation (the `TF` prefix) of the Pytorch `DistilBert` model
- with a classification head for `SequenceClassification`


## Model card

Part of a good model hub is a *model card* for each model that describes
- The model
- The data on which it was trained
- Gives reference to the original work (paper, code)
- Discusses potential *bias*

There is often also some very practical (technical) information
- code sample
- how long it took to train 
    - 90 hours using 8 GPU's
    - we will get all the benefit without the cost of training via Fine-Tuning

A potential user can also often get a sense of the behavior via a *Hosted Inference API*
- try the model without coding !
 - The `Deploy` button gives code fragments for
     - accessing this hosted inference
     - creating *your own* hosted inference
         - free version: limited host machine capability
         - paid versions
 
[Here](https://huggingface.co/distilbert-base-uncased) is the model card for `DistilBert`.  

## Model bias

Bias of models (really, the data on which they were trained) is important and is being increasingly recognized.

Models are often trained from publicly available Web content (e.g., Wikipedia).

Although the datasets were not designed to be biased
- the frequency of biased concepts that they contain
- causes the biases to be transfered to models on which they are trained.

In the Intro course, we discussed how the GloVe word embeddings may reveal bias:

$
\begin{array}[llllll]\\
\text{doctor - man + woman} &  \approx_{n',d} & \text{nurse } \\
\text{mechanic  - man + woman} &  \approx_{n',d} & \text{teacher } \\
\end{array}
$

An important part of releasing a model for public use is informing the user of any bias that they inherit.

Discovering and documenting potential bias is non-trivial and potentially time-consuming.

In this case, for the Masked Language Modeling task, the authors look at what the model chooses
- to fill in the mask
- conditional on some aspects of the subject of the sentence

[Here are the biases](https://huggingface.co/distilbert-base-uncased#limitations-and-bias) discussed on the model card.

# Using a pre-trained model for NLP (w/o Transfer Learning)

Our first demo will be to show how to use a pre-trained model.
- We will not (yet) specialize it to a new Task

The task is Text Sequence Classification (sentiment)

The model is based on the Transformer architecture.

We give a quick review of how Transformers are typically used for NLP.

**Pre processing**

Text is a sequence of words, but the words must be parsed into *tokens*
- a word may turn into multiple tokens: Byte Piece Encoding

The tokens must be converted into integer indices in a finite vocabulary
- It is the sequence of *token ids* that is consumed by the Transformer

**Post processsing**

Hugging Face Classification models are slightly different in that they return
- logits (scores)
- **not** probabilities

We can use `argmax` on the score to decide the class, but it's nice to see the associated probability too.

## Making it concrete: let's write some code

We make these abstract ideas concrete by writing some code
- especially: looking at the shape and type outputs

Let's go to the [notebook](HF_quick_intro_to_models.ipynb)   

In [2]:
print("Done")

Done
